# Deep learning survival methods benchmarks on ALS

In questo notebook vorrei raccogliere tutti i benchmark che abbiamo fatto Corrado ed io e uniformarli
- per semplificare usiamo solo dataset staticizzati, tanto non ci sono un'enormita' di feature dinamiche per la maggior parte dei pazienti
- mi piacerebbe investigare se il modo di trattare gli eventi (indipendenti, competing, multi-state) ha impatto sulla predizione
- stiamo attenti al brier score e ai metodi che usano ipcw perche' in questo dataset mi pare faccia un casino per tempi bassi
- bisogna decidere una singola metrica per ottimizzare i modelli, anche se poi ne calcoliamo varie
- i tempi di valutazione li prendiamo come quantili dei tempi degli eventi globali? o dal dataset di train?

- survtrace: sembra che ci sia un trattamento particolare per le feature categoriche, al momento gliele passiamo OHE come numeriche, forse non e' la cosa migliore

### Cose da fare
- provare ad aggiungere soden?
- 

## Datasets
Staticized datasets at first ALSFRS and after six months

There are three events: NIV, PEG and death.

In [1]:
import numpy
import pandas
import seaborn
import matplotlib.pyplot as plt

In [2]:
#assert Xdiag.index.equals(event_df.index)
#Ydiag = {
#    e: numpy.array(
#        event_df[[e + '_event', e + '_time']].apply(tuple, axis=1), 
#        dtype=[('event', bool), ('time', float)]
#    ) for e in ['death', 'NIV', 'PEG']
#}

In [3]:
#Y6m = numpy.array(
#    pandas.read_csv('death6_Y.csv', index_col='id').apply(tuple, axis=1),
#    dtype=[('event', bool), ('time', float)]
#)
#Y6m['time'] -= 6

### Train-test split and final preprocessing before prediction 

In [4]:
from sklearn.impute import SimpleImputer
from sklearn.model_selection import KFold, GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [5]:
def to_numpy(*args):
    return (a.values if hasattr(a, 'values') else a for a in args)

In [6]:
def preproc_for_prediction(X, time, event, max_miss=0.3, seed=0):
    Xclean = X[:, numpy.isnan(X).mean(axis=0) < max_miss]
    # shift times to make them positive
    if numpy.min(time) == 0:
        time = time.copy()
        time += numpy.min(time[time > 0])/3
        
    X_train, X_test, time_train, time_test, event_train, event_test = train_test_split(Xclean, time, event, test_size=0.2, random_state=seed)
    assert all(numpy.nanstd(X_train, axis=0) > 0)
    scaler  = StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train)
    X_test  = scaler.transform(X_test)
    imputer = SimpleImputer(strategy='median').fit(X_train)
    X_train = imputer.transform(X_train)
    X_test  = imputer.transform(X_test)
    return dict(train=(X_train, time_train, event_train), test=(X_test, time_test, event_test))

In [7]:
def surv_vector(times, events):
    assert times.shape == events.shape
    return numpy.array(
        list(zip(*to_numpy(events, times))), 
        dtype=[('event', bool), ('time', float)]
    )
def surv_matrices(times, events):
    assert times.shape == events.shape
    times, events = to_numpy(times, events)
    return [
        surv_vector(times[:, evt], events[:, evt])
        for evt in range(times.shape[1])
    ]

## Helper functions
Encapsulate each different method into a class that generate the model from an Optuna trial object and 

In [8]:
if False:
    class SurvMethod:
        def __init__(self, trial):
            self.trial = trial
            self.model_params = trial2model_params(trial)
            self.fit_params = {}
        @staticmethod
        def trial2model_params(trial):
            raise NotImplemented()
        @staticmethod
        def trial2fit_params(trial):
            return {}
        def train(self, X, Y):
            self.model = self.trial2model(self.trial)
            self.model.fit(X, y, **trial2fit_params(self.trial))
        def predict(self, X):
            self.model.predict(X)

In [9]:
import json
def hash_dict(d):
    return json.dumps(dict(sorted(d.items())))

In [10]:
import gzip
import pickle

def my_open(path, mode):
    return (gzip.open if path.endswith('.gz') else open)(path, mode)

def load_from(path):
    with my_open(path, 'rb') as f:
        return pickle.load(f)

def dump_to(obj, path):
    with my_open(path, 'wb') as f:
        pickle.dump(obj, f)

def load_or_new(func, path):
    try:
        return load_from(path)
    except FileNotFoundError:
        return func()

In [11]:
def event_as_bool(event):
    return event if event.dtype == 'bool' else event == 1
def get_time_quantiles(*y, q):
    if len(y) == 2:
        time, event = y
        event = event_as_bool(event)
        if len(time.shape) == 1:
            return numpy.quantile(time[event], q)
        return numpy.array([numpy.quantile(time[:,evt][event[:,evt]], q) for evt in range(time.shape[1])])
    elif len(y) == 1:
        y, = y
        return numpy.quantile(y['time'][y['event']], q)

In [12]:
def my_brier(y, preds, t):
    keep = (y['time'] > t) | y['event']
    y_bin = y['time'][keep] <= t
    return numpy.mean(numpy.square(preds[keep] - y_bin))

## Cross-validation and evaluation

Evaluate method in cross-validation, return average c-index across cross-validation set, event and evaluation time

In [13]:
class FailedModel(Exception):
    pass

In [14]:
def check_probs(model, p):
    nans = numpy.sum(numpy.isnan(p))
    if nans:
        raise FailedModel(f'{model.short_name} model bad prediction: found {nans} nans')
    unders = numpy.sum(p < 0.0)
    if unders:
        raise FailedModel(f'{model.short_name} model bad prediction: found {unders} values greater than 1.0')
    overs = numpy.sum(p > 1.0)
    if overs:
        raise FailedModel(f'{model.short_name} model bad prediction: found {overs} values greater than 1.0')

In [15]:
# redefinition to handle competing events where the event is expressed as an integer
# 0 is censoring, 1 is the main event and values greater than 1 are other competing events that we treat as censoring when testing performance
def concordance_index_censored(event, time, preds):
    from sksurv.metrics import concordance_index_censored as cic
    return cic(event == 1, time, preds)    

In [16]:
class Silence:
    def __enter__(self, stdout=True, stderr=True):
        null_output = open('/dev/null', 'wt')
        if stdout:
            self.original_stdout = sys.stdout
            sys.stdout = null_output
        else:
            self.original_stdout = None
        if stderr:
            self.original_stderr = sys.stderr
            sys.stderr = null_output
        else:
            self.original_stderr = None
            
        sys.stdout
    def __exit__(self, *_):
        if self.original_stdout is not None:
            sys.stdout = self.original_stdout
        if self.original_stderr is not None:
            sys.stdout = self.original_stderr
            
        return False

In [17]:
if False:
    from contextlib import contextmanager

    @contextmanager
    def silence(stdout=True, stderr=True):
        import sys
        if stdout:
            original_stdout = sys.stdout
            sys.stdout
        resource = acquire_resource(*args, **kwds)
        try:
            yield resource
        finally:
            # Code to release resource, e.g.:
            release_resource(resource)


In [18]:
import sksurv.metrics
import optuna

def generic_objective_function(method, X, time, event, eval_times, cv=KFold(n_splits=5)):
    cv_scores = []
    assert len(eval_times.shape) == 1
    
    if len(time.shape) == 2: # for multi event only evaluate first (main) event
        #assert time.shape[1] == eval_times.shape[0]
        main_time = time[:, 0]
        main_event = event[:, 0]
        #eval_times = eval_times[0]
        #print('multi')
    else:
        main_time = time
        main_event = event
    
    try:
        for train_idx, test_idx in cv.split(X):
            # silence model output            
            #with Silence():
            method.fit(X[train_idx], time[train_idx], event[train_idx])
            preds = method.predict(X[test_idx], eval_times)
            check_probs(method, preds)

            assert preds.shape == (len(test_idx), len(eval_times)), f'{preds.shape} != ({len(test_idx)}, {len(eval_times)}) (sample, eval_time)'
            for ti, t in enumerate(eval_times):
                #print(main_event[test_idx].shape, main_time[test_idx].shape, preds[:, ti].shape)
                s = concordance_index_censored(main_event[test_idx], main_time[test_idx], preds[:, ti])[0]
                cv_scores.append(s)
    except FailedModel as e:
        print(f'Failed trial: {e}')
        raise optuna.TrialPruned(str(e))
    return numpy.mean(cv_scores)

In [19]:
def evaluate_pred(
    model, # trained model
    datasets, # train and test datasets
    eval_times):
    
    assert len(eval_times.shape) == 1, 'for now only evaluate first (main) event'

    X_test, time_test, event_test = datasets['test']
    _, time_train, event_train = datasets['train']
    
    preds = model.predict(X_test, eval_times)

    if len(time_train.shape) == 2:
        # select only first (main) event
        if len(preds.shape) == 3:
            assert preds.shape[1] == time_train.shape[1]
            preds = preds[:, 0]
        time_test, event_test, time_train, event_train = (
            x[:, 0] for x in [time_test, event_test, time_train, event_train])
        
        
    
    
    #competing = numpy.any(event_train == 2)
    event_test = event_as_bool(event_test)
    
    y = surv_vector(time_test, event_test)
    y_train = surv_vector(time_train, event_train)
    acc = {}
    
    max_censor_time = numpy.max(y['time'][~y['event']])
    for ti, t in enumerate(eval_times):
        p = preds[:, ti]
        assert t < max_censor_time
        acc[('concordance_index_censored', ti, t)] = concordance_index_censored(y['event'], y['time'], p)[0]
        acc[('cumulative_dynamic_auc', ti, t)] = sksurv.metrics.cumulative_dynamic_auc(y_train, y, p, [t])[1]
        acc[('raw_brier_score', ti, t)] = my_brier(y, p, t)
        acc[('concordance_index_ipcw', ti, t)] = sksurv.metrics.concordance_index_ipcw(y_train, y, p, tau=max_censor_time)[0]
        acc[('concordance_index_ipcw_t', ti, t)] = sksurv.metrics.concordance_index_ipcw(y_train, y, p, tau=t)[0]
        #acc[('brier_score', ti, t)] = sksurv.metrics.brier_score(y_train, y, 1 - p, [t])[1][0]
        
    scores = pandas.Series(acc, name='VALUE', dtype=float)
    scores.index.names = ['METRIC', 'ITIME', 'TIME']
    return scores

In [20]:
import optuna

def optimize_model(method, X, time, event, max_trials=None, study_name=None, time_quantiles=[0.1, 0.25, 0.5, 0.75, 0.9], seed=0, skip_eval=False):
    X, time, event = to_numpy(X, time, event)
    eval_times = get_time_quantiles(time, event, q=time_quantiles)
    train_test_ds = preproc_for_prediction(X, time, event, seed=seed)

    if len(time.shape) == 2: # multi state
        assert eval_times.shape[0] == time.shape[1]
        eval_times = eval_times[0] # only evaluate first (main) event
    
    #global study
    study = optuna.create_study(
        direction='maximize', sampler=optuna.samplers.RandomSampler(seed=seed), study_name=study_name, load_if_exists=True,
        storage=(f'sqlite:///{work_dir}/{study_name}.optuna.sqlite3' if study_name is not None else None))
    
    def count_complete():
        return sum(t.state == optuna.trial.TrialState.COMPLETE for t in study.trials)
    
    from collections import Counter
    trial_counts = dict(Counter(str(t.state).split('.')[1] for t in study.trials))
        
    if True:
        print('Loaded', len(study.trials), 'trials:', ', '.join(f'{n} {s}' for s, n in trial_counts.items()))

    if max_trials is None or count_complete() < max_trials:
        print(count_complete(), 'of', max_trials, 'completed')
    while max_trials is None or count_complete() < max_trials:
        study.optimize(lambda trial: generic_objective_function(method(trial), *train_test_ds['train'], eval_times), n_trials=1)
    
    if skip_eval:
        return pandas.Series(dtype='float64'), trial_counts
    try:
        best_trial = study.best_trial
    except ValueError:
        print('No trials to evaluate')
        return pandas.Series(dtype='float64'), trial_counts
    
    #print('Test best model')
    model = None
    if study_name is not None:
        model_key = hash_dict(best_trial.params)
        model_cache_path = f'{work_dir}/{study_name}.models.pickle.gz'
        try:
            model_cache = load_or_new(dict, model_cache_path)
        except RuntimeError as e: # this could be due to a torch model saved on an unavailable device
            print('Error loading model (will be retrained):', e)
            model_cache = {}
        model = model_cache.get(model_key)
        
    
    if model is None:
        #print('Train best model')
        model = method(best_trial).fit(*train_test_ds['train'])
        if study_name is not None:
            model_cache[model_key] = model
            dump_to(model_cache, model_cache_path)

    return evaluate_pred(model, train_test_ds, eval_times), trial_counts

## Methods

In [21]:
#import sys
#sys.path.append('../')
from utils import *

In [22]:
class SurvMethod:
    independent = True
    
    @staticmethod
    def trial2model_params(trial):
        raise NotImplementedError()
    def __init__(self, trial):
        self.trial = trial
        self.model_params = self.trial2model_params(trial)
    def fit(self, X, times, events):
        """X: sample, feat
        times: sample, event
        events: sample, event
        """
        raise NotImplementedError()
    def predict(self, X, eval_times):
        """predict probabilites of event up to given times for each event, with shape (sample, event, eval_time)"""
        raise NotImplementedError()

### Scikit Survival Methods

In [23]:
class SkSurvMethod(SurvMethod):
    """Generic wrapper for a Scikit Survival model"""
    model_func = None
    competing = False
    multi = False

    def fit(self, X, times, events):
        y = surv_vector(times, events)
        #print(y.shape, y)
        self.model_ = self.model_func(**self.model_params).fit(X, y)
        return self
    
    def predict(self, X, eval_times):
        """predict probabilites of event up to given times for each event"""
        return numpy.array([[1 - sf(t) for t in eval_times] for sf in self.model_.predict_survival_function(X)])        


In [24]:
class SkSurvMultiMethod(SurvMethod):
    """This class is similar to SkSurvMethod but creates a Scikit Survival model with the same parameters for each event independently"""
    model_func = None

    def fit(self, X, times, events):
        Y = surv_matrices(times, events)
        self.models_ = []
        for y in Y:
            m = self.model_func(**self.model_params)
            m.fit(X, y)
            self.models_.append(m)
        return self
    def predict(self, X, eval_times):
        """predict probabilites of event up to given times for each event"""
        return numpy.swapaxes([[[1 - sf(t) for t in tt] for sf in m.predict_survival_function(X)] for tt, m in zip(eval_times, self.models_)], 0, 1)

In [25]:
class SkCoxPH(SkSurvMethod):
    short_name = 'skcoxph'
    long_name = 'Cox PH'
    from sksurv.linear_model import CoxPHSurvivalAnalysis
    model_func = CoxPHSurvivalAnalysis
    
    @staticmethod
    def trial2model_params(trial):
        return dict(
            alpha=trial.suggest_float('alpha', 0, 10),
            ties=trial.suggest_categorical('ties', ["breslow", "efron"]),
            n_iter=10000, tol=1e-09, verbose=0,
        )

In [26]:
class SkRSF(SkSurvMethod):
    short_name = 'skrsf'
    long_name = 'Random Survival Forest'

    from sksurv.ensemble import RandomSurvivalForest
    model_func = RandomSurvivalForest
    
    @staticmethod
    def trial2model_params(trial):
        return dict(
            n_estimators = 1000, #trial.suggest_int('n_estimators', 100, 1000, 100),
            max_depth    = trial.suggest_int('max_depth ', 1, 5),
            n_jobs=n_jobs,
        )

### Deep Survival Machines

In [27]:
import sys
sys.path.append('./auton-survival')

In [28]:
if False:
    def generate_layer_sizes(trial, min_hidden_layers=0, max_hidden_layers=4, max_log2_size=8, prefix=''):
        layers = []
        for i in range(trial.suggest_int(f'{prefix}n_hidden', min_hidden_layers, max_hidden_layers)):
            size = trial.suggest_int(f'{prefix}log2_layer_{i}', 0, max_log2_size)
            layers.append(2**size)
            max_log2_size = size
        return layers

In [29]:
class DeepSurvivalMachines(SurvMethod):
    short_name = 'dsm'
    long_name = 'Deep Survival Machines'
    competing = True
    multi = False
    
    @staticmethod
    def trial2model_params(trial):
        return dict(
            mod = dict(
                k             = trial.suggest_int('k', 1, 6),
                distribution  = trial.suggest_categorical('distribution', ['LogNormal', 'Weibull']),
                layers        = generate_layer_sizes(trial, max_hidden_layers=4, max_log2_size=8),
            ),
            fit = dict(
                learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2, log=True),
                vsize         = trial.suggest_float('val_size', 0.05, 0.5, log=True),
                batch_size    = 2**trial.suggest_int('batch_size', 2, 10),
                elbo          = trial.suggest_categorical('elbo', [True, False]),
                optimizer     = trial.suggest_categorical('optimizer', ['Adam', 'RMSProp', 'SGD']),
                iters         = 2**trial.suggest_int('max_epochs', 3, 10),
            )
        )

    def fit(self, X, times, events):
        import sys
        
        from auton_survival.models.dsm import DeepSurvivalMachines
        try:
            self.model_ = DeepSurvivalMachines(
                **self.model_params['mod']).fit(
                X, times, events, **self.model_params['fit'])
        except RuntimeError as e:
            raise FailedModel(f'{self.short_name} model fit failed: {e}')
        return self

    def predict(self, X, eval_times):
        """predict probabilites of event up to given times for each event"""
        #global dbg
        #dbg = self.model_, X, eval_times, numpy.swapaxes([self.model_.predict_risk(X, t)[:, 0] for t in eval_times], 0, 1)
        return numpy.swapaxes([self.model_.predict_risk(X, t)[:, 0] for t in eval_times], 0, 1)
        #return numpy.nan_to_num(numpy.swapaxes([self.model_.predict_risk(X, t)[:, 0] for t in eval_times], 0, 1), nan=0.5, posinf=1, neginf=0)

### pycox models

In [30]:
import sys
sys.path.insert(0, './pycox')
sys.path.insert(0, './torchtuples')
from pycox.preprocessing.label_transforms import LabTransDiscreteTime
import torch
import torchtuples

In [31]:
class LabTransformCompeting(LabTransDiscreteTime):
    def transform(self, durations, events):
        durations, is_event = super().transform(durations, events > 0)
        events[is_event == 0] = 0
        return durations, events.astype('int64')

class CauseSpecificNet(torch.nn.Module):
    """Network structure similar to the DeepHit paper, but without the residual
    connections (for simplicity).
    """
    def __init__(self, in_features, num_nodes_shared, num_nodes_indiv, num_risks,
                 out_features, batch_norm=True, dropout=None, residual=False):
        super().__init__()
        self.residual = residual
        self.shared_net = torchtuples.practical.MLPVanilla(
            in_features, num_nodes_shared[:-1], num_nodes_shared[-1],
            batch_norm, dropout,
        )
        self.risk_nets = torch.nn.ModuleList()
        for _ in range(num_risks):
            net = torchtuples.practical.MLPVanilla(
                num_nodes_shared[-1] + (in_features if residual else 0), 
                num_nodes_indiv, out_features,
                batch_norm, dropout,
            )
            self.risk_nets.append(net)

    def forward(self, input):
        out = self.shared_net(input)
        if hasattr(self, 'residual') and self.residual:
            out = torch.cat([out, input], dim=-1)
        out = [net(out) for net in self.risk_nets]
        out = torch.stack(out, dim=1)
        return out

In [32]:
class DeepHitMethod(SurvMethod):
    short_name = 'deephit'
    long_name = 'DeepHit'
    multi = False
    competing = True
    residual = False
    
    @staticmethod
    def trial2model_params(trial):
        layer_sizes  = generate_layer_sizes(trial, min_hidden_layers=1, max_hidden_layers=4, max_log2_size=8)
        shared_layers_num = trial.suggest_int('shared_layers_num', 1, len(layer_sizes))
        
        return dict(
            num_durations = trial.suggest_int('num_durations', 10, 50, step=10),

            net = dict(
                batch_norm = trial.suggest_categorical('batch_norm', [True, False]),
                dropout    = trial.suggest_float('dropout', 0.01, 0.5, log=True),
            ),
            indepnet = dict(
                num_nodes  = layer_sizes,
            ),
            compnet = dict(
                num_nodes_shared = layer_sizes[:shared_layers_num],
                num_nodes_indiv  = layer_sizes[shared_layers_num:],
                #residual         = trial.suggest_categorical('batch_norm', [True, False]),
            ),
            mod = dict(
                alpha                  = trial.suggest_float('alpha', 1e-3, 1e-2, log=True),
                sigma                  = trial.suggest_float('sigma', 1e-2, 1, log=True),
            ),
            opt = dict(
                lr                     = trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True),
                decoupled_weight_decay = trial.suggest_float('weight_decay', 1e-4, 1e-2, log=True),
                cycle_eta_multiplier   = trial.suggest_float('eta_multiplier',0.2,1),
            ),
            fit = dict(
                epochs        = 2**trial.suggest_int('epochs_log2', 2, 5),
                batch_size    = 2**trial.suggest_int('batch_log2', 3, 8),
            ),
        )

    def fit(self, X, times, events):
        from pycox.models import DeepHitSingle, DeepHit

        assert len(events.shape) == 1
        num_risks = numpy.max(events.astype(float))
        assert num_risks == int(num_risks)
        num_risks = int(num_risks)
        self.num_risks_ = num_risks
        
        optimizer = torchtuples.optim.AdamWR(**self.model_params['opt'])
        #print('fit', num_risks, times.shape, events.shape)
        if num_risks == 1:
            self.labtrans_ = DeepHitSingle.label_transform(self.model_params['num_durations'])
            y = self.labtrans_.fit_transform(times, events)
            net = torchtuples.practical.MLPVanilla(
                in_features=X.shape[1], out_features=self.labtrans_.out_features, 
                **self.model_params['indepnet'], **self.model_params['net'])
            method = DeepHitSingle
        else:
            self.labtrans_ = LabTransformCompeting(self.model_params['num_durations'])
            y = self.labtrans_.fit_transform(times, events)
            net = CauseSpecificNet(
                in_features=X.shape[1], out_features=self.labtrans_.out_features, num_risks=num_risks, 
                **self.model_params['compnet'], **self.model_params['net'], residual=self.residual)
            method = DeepHit
        
        self.model_ = method(net, optimizer, **self.model_params['mod'], device=torch_device, duration_index=self.labtrans_.cuts)
        self.model_.fit(X.astype('float32'), y, num_workers=0 if True else n_jobs, verbose=False, **self.model_params['fit'])
        return self

    def predict(self, X, eval_times):
        preds = 1 - self.model_.predict_surv(X.astype('float32'))
        if hasattr(self, 'num_risks_') and self.num_risks_ > 1:
            preds = preds.T
        #print('predict', eval_times.shape, self.labtrans_.cuts.shape, preds.shape)
        return numpy.array([numpy.interp(eval_times, self.labtrans_.cuts, p, left=0, right=1) for p in preds])
#test_methods([DeepHitMethod])

In [33]:
class DeepHitMethodRes(DeepHitMethod):
    short_name = 'deephitres'
    long_name = 'DeepHitRes'
    residual = True
    independent = False
#test_methods([DeepHitMethodRes])

In [34]:
class DeepSurvMethod(SurvMethod):
    short_name = 'deepsurv'
    long_name = 'DeepSurv'
    multi = False
    competing = False
    
    @staticmethod
    def trial2model_params(trial):
        return dict(
            net = dict(
                num_nodes  = generate_layer_sizes(trial, min_hidden_layers=1, max_hidden_layers=4, max_log2_size=8),
                batch_norm = trial.suggest_categorical('batch_norm', [True,False]),
                dropout    = trial.suggest_float('dropout', 0.01, 0.5, log=True),
            ),
            opt = dict(
                lr                     = trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True),
                decoupled_weight_decay = trial.suggest_float('weight_decay', 1e-4, 1e-2, log=True),
                cycle_eta_multiplier   = trial.suggest_float('eta_multiplier',0.2,1),
            ),
            fit = dict(
                epochs        = 2**trial.suggest_int('epochs_log2', 2, 5),
                batch_size    = 2**trial.suggest_int('batch_log2', 3, 8),
            ),
        )

    def fit(self, X, times, events):
        from pycox.models import CoxPH
        
        optimizer = torchtuples.optim.AdamWR(**self.model_params['opt'])

        self.utimes_ = sorted(set(times))
        
        net = torchtuples.practical.MLPVanilla(
            in_features=X.shape[1], out_features=1, output_bias=False,
            **self.model_params['net'],
        )
        
        self.model_ = CoxPH(net, optimizer, device=torch_device)#, **self.model_params['mod'], device=torch_device, duration_index=self.labtrans_.cuts)
        self.model_.fit(X.astype('float32'), (times, events), num_workers=0 if True else n_jobs, verbose=False, **self.model_params['fit'])
        self.model_.compute_baseline_hazards()
        
        return self

    def predict(self, X, eval_times):
        preds = 1 - self.model_.predict_surv(X.astype('float32'))
        return numpy.array([numpy.interp(eval_times, self.utimes_, p, left=0, right=1) for p in preds])
#test_methods([DeepSurvMethod])

### SurvTRACE

In [35]:
import sys
sys.path.append('./SurvTRACE')
sys.path.append('./easydict')

In [36]:
if False:
    # prova per capire i competing events
    from survtrace import STConfig
    from survtrace.dataset import load_data
    STConfig['data']='seer'
    load_data(STConfig)
    assert False

In [37]:
#dbg = None
class SurvTraceMethod(SurvMethod):
    short_name = 'survtrace'
    long_name = 'SurvTRACE'
    multi = False
    competing = False # TODO add competing!
    
    @staticmethod
    def trial2model_params(trial):
        # Got the error: "The hidden size (16) is not a multiple of the number of attention heads (6)"
        # So the hidden size is now a multiple of the attention head number
        num_attention_heads  = trial.suggest_int('num_attention_heads', 2, 6)
        return dict(
            conf = dict(
                hidden_size          = num_attention_heads*trial.suggest_int('hidden_size_log2', 1, 6), 
                intermediate_size    = 2**trial.suggest_int('intermediate_size_log2', 3, 7),
                hidden_dropout_proba = trial.suggest_float('hidden_dropout_proba', 1e-3, 0.5, log=True),
                num_hidden_layers    = trial.suggest_int('num_hidden_layers', 1, 4),
                num_attention_heads  = num_attention_heads,
            ),
            fit = dict(
                learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True),
                epochs        = 2**trial.suggest_int('epochs_log2', 2, 6),
                batch_size    = 2**trial.suggest_int('batch_size_log2', 3, 8),
                weight_decay  = trial.suggest_float('weight_decay', 1e-4, 1e-2, log=True),
            ),
        )

    def fit(self, X, times, events):
        from survtrace.utils import LabelTransform
        from survtrace import STConfig, Trainer
        from survtrace.model import SurvTraceSingle
        
        assert len(events.shape) == 1
        num_risks = numpy.max(events.astype(float))
        assert num_risks == int(num_risks)
        num_risks = int(num_risks)

        # data preprocessing
        self.labtrans_ = LabelTransform(cuts=numpy.array(
            [times.min()] +
            numpy.quantile(times[events == 1], STConfig['horizons']).tolist() +
            [times.max()]
        ))
        self.labtrans_.fit(times, events)
        y = self.labtrans_.transform(times, events)
        
        if num_risks == 1:
            ydf = pandas.DataFrame({"duration": y[0], "event": y[1], "proportion": y[2]})
        else:
            print('num_risks:', num_risks)
            ydf = pandas.DataFrame({"duration": y[0], "proportion": y[2]})
            for evt in range(num_risks):
                ydf['event_' + str(evt)] = (events == (evt + 1)).astype(float)
            print(pandas.concat([ydf, pandas.DataFrame({'events': events, 'y[1]': y[1]})], axis=1).head(20))
                

        STConfig['labtrans'] = self.labtrans_
        STConfig['num_numerical_feature'] = X.shape[1]
        STConfig['num_categorical_feature'] = 0 #int(len(cols_categorical))
        STConfig['num_feature'] = X.shape[1]
        STConfig['vocab_size'] = 0
        STConfig['duration_index'] = self.labtrans_.cuts
        STConfig['out_feature'] = int(self.labtrans_.out_features)
        
        STConfig['num_event'] = num_risks
        
        STConfig.update(self.model_params['conf'])
        self.model_ = SurvTraceSingle(STConfig)
        
        # initialize a trainer
        trainer = Trainer(self.model_)
        train_loss, val_loss = trainer.fit((pandas.DataFrame(X.astype('float32')), ydf), **self.model_params['fit'])
        #global dbg
        #dbg = self, X, y
        return self

    def predict(self, X, eval_times):
        preds = 1 - self.model_.predict_surv(pandas.DataFrame(X.astype('float32')))
        return numpy.array([numpy.interp(eval_times, self.labtrans_.cuts, p, left=0, right=1) for p in preds.cpu()])
#test_methods([SurvTraceMethod], event_mode='competing')

### Modello mio

In [38]:
class InterNN(torch.nn.Module):
    def __init__(self, input_size, output_size, interaction_size, hidden_activation=torch.nn.ReLU, **kwargs):
        super().__init__()
        self.enc1 = FFN(input_size, interaction_size, output_activation=hidden_activation, **kwargs)
        self.enc2 = FFN(input_size, interaction_size, output_activation=hidden_activation, **kwargs)
        self.ffn = FFN(interaction_size**2, output_size, hidden_activation=hidden_activation, **kwargs)
    def forward(self, x):
        l1 = self.enc1(x).unsqueeze(-1)
        l2 = self.enc2(x).unsqueeze(-2)
        ll = (l1*l2).flatten()
        return self.ffn(ll)

In [39]:
class SURVTORCH(torch.nn.Module):
    def __init__(
            self, 
            input_size,
            times,
            interaction_size=None,
            **ffn_kwargs,
        ):
        super().__init__()
        self.times = times
        self.delta_times = times[1:] - times[:-1]
        assert len(self.times.shape) == 2
        output_size = self.times.shape[0]*self.times.shape[1]
        if interaction_size is None:
            self.net = FFN(
                input_size, output_size, 
                output_activation=None,
                **ffn_kwargs,
            )
        else:
            self.net = InterNN(
                input_size, output_size, 
                interaction_size=interaction_size,
                output_activation=None,
                **ffn_kwargs,
            )

        
    def forward(self, x, times=None):
        # times should be 1-dimensional!
        y = self.net(x).reshape(-1, *self.times.shape)
        y = torch.nn.functional.softmax(y, 1)
        if times is None:
            return y

        acc = []
        for t in times:                
            interval_weights = torch.clamp((t - self.times[:-1])/self.delta_times, 0, 1)
            inf_weigths = 1 - 2**-torch.clamp(t/self.times[-1] - 1, 0)
            weights = torch.cat([interval_weights, inf_weigths.reshape(1, -1)])
            survival = torch.sum(y*weights.reshape(1, *weights.shape), -2)
            acc.append(survival.reshape(-1, self.times.shape[1], 1))
        return torch.cat(acc, dim=2)

In [40]:
# maybe move most of this code to SURVTORCH
from tqdm import tqdm
import sklearn
#from sklearn.base import BaseEstimator, ClassifierMixin
import dataclasses

def time_quantiles(y_times, y_events, quantiles):
    try:
        quantiles = numpy.linspace(0, 1, quantiles + 1)
    except TypeError:
        pass
    return numpy.array([
        numpy.quantile(y_times[y_events[:, i], i], quantiles)
        for i in range(y_times.shape[1])
    ]).T

@dataclasses.dataclass(eq=False)#, kw_only=True)
class GBSURV(sklearn.base.BaseEstimator):
    time_bins: int = 10
    epochs: int = 10
    learning_rate: float = 1e-4
    batch_size: int = 64
    ffn_params: dict = dataclasses.field(default_factory=dict)
    interaction_size: int = None
    device: str = 'cpu'
    #hidden_sizes: list = dataclasses.field(default_factory=list)
    #dropout: float = 0.1
    #hidden_activation=torch.nn.ReLU    
    
    def as_tensor(self, x): # maybe move this to SURVTORCH
        return torch.from_numpy(x.astype(numpy.float32)).to(self.device)
        
    def fit(self, X, y_times, y_events, X_test=None, y_test_times=None, y_test_events=None, test_times=None):
        assert numpy.any(y_events), f'no event in {y_events.shape}'
        self.time_breaks_ = time_quantiles(y_times, y_events, self.time_bins)
        
        model = SURVTORCH(
            input_size=X.shape[1], 
            times=self.as_tensor(self.time_breaks_),
            **self.ffn_params,
        ).to(self.device)

        dl = torch.utils.data.DataLoader(
            torch.utils.data.TensorDataset(
                self.as_tensor(X), self.as_tensor(y_times),
                torch.from_numpy(y_events.astype(bool)).to(self.device),
            ), 
            batch_size=self.batch_size,
        )
        
        opt = torch.optim.Adam(model.parameters(), lr=self.learning_rate)
        
        
        times0 = y_times[y_events] # sample times
        if times0.shape[0] == 0:
            print(y_times, y_times.shape)
            print(times0, times0.shape)
            print(y_events, y_events.shape)
        assert times0.shape[0] > 0, f'sampling times are empty: {times0}'
        
        self.train_history = []
        if X_test is not None:
            if test_times is None:
                test_times = time_quantiles(y_test_times, y_test_events, [0.25, 0.50, 0.75])
            X_test = self.as_tensor(X_test)
            y_test_times = self.as_tensor(y_test_times)
            y_test_events = torch.from_numpy(y_test_events.astype(bool))
        
        
        for epoch in tqdm(range(self.epochs)):
            #print(f"Epoch: {epoch + 1}")
            model.train()
            train_losses = []
            epoch_history = {'train losses': []}
            for X, y_times, y_events in dl:
                # chose random time for training
                t = times0[torch.randint(0, times0.shape[0], ())]

                p = model(X, [t])[:, :, 0]
                
                informative = ((y_times > t) | y_events).to(torch.float32)
                events = ((y_times <= t) & y_events).to(torch.float32)
                loss = torch.nn.functional.mse_loss(
                    p*informative,
                    events,
                )


                opt.zero_grad()
                loss.backward()
                opt.step()
                epoch_history['train losses'].append(loss.item())
            if X_test is not None:
                epoch_history['test errors'] = []
                model.eval()
                err_table = []
                with torch.no_grad():
                    for event_idx, event_times in enumerate(test_times.T):
                        p = model(X_test, event_times)
                        event_errs = []
                        for i, t in enumerate(event_times):
                            p0 = p[:, event_idx, i]
                            informative = ((y_test_times[:, event_idx] > t) | y_test_events[:, event_idx]).to(torch.float32)
                            events = ((y_test_times[:, event_idx] <= t) & y_test_events[:, event_idx]).to(torch.float32)
                            #print(p0.shape, informative.shape, events.shape)
                            loss = torch.nn.functional.mse_loss(
                                p0*informative,
                                events,
                            )
                            informative = (y_test_times[:, event_idx] > t) | y_test_events[:, event_idx]
                            events = (y_test_times[:, event_idx] <= t) & y_test_events[:, event_idx]
                            score = torch.nn.functional.mse_loss(p0[informative], events[informative])
                            #print(informative.shape)
                            #assert False
                            event_errs.append(score.item())
                            #print(loss)
                        epoch_history['test errors'].append(event_errs)
            self.train_history.append(epoch_history)
                
        self.model_ = model
        
        return self
    
    def predict(self, X, times=None):
        self.model_.eval()
        return self.model_(self.as_tensor(X), times).detach().cpu().numpy()
        
#from sklearn.utils.estimator_checks import check_estimator
#check_estimator(SURV())
#est = SURV(arch='MMM', time_bins=10, epochs=200).fit(X_train, y_train_times, y_train_events, X_test, y_test_times, y_test_events)
#est = SURV(arch='MMM', time_bins=10, epochs=2000, hidden_sizes=[64, 16], nn_params=dict(embed_size=64, dropout=0.1)).fit(X_train, y_train_times, y_train_events, X_test, y_test_times, y_test_events)
#est

In [41]:
class GBM(SurvMethod):
    short_name = 'gbm'
    long_name = 'gbm'
    multi = True
    competing = False
    
    @staticmethod
    def trial2model_params(trial):
        return dict(
            time_bins = trial.suggest_int('num_durations', 3, 30),
            epochs        = 2**trial.suggest_int('epochs_log2', 2, 5),
            batch_size    = 2**trial.suggest_int('batch_log2', 3, 8),
            learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True),

            ffn_params = dict(
                hidden_sizes      = generate_layer_sizes(trial, max_hidden_layers=4, max_log2_size=8),
                dropout           = trial.suggest_float('dropout', 0.01, 0.5, log=True),
                hidden_activation = suggest_activation(trial, 'activation'),
            ),
        )

    def fit(self, X, times, events):
        
        if len(times.shape) == 1:
            times = times[:, None]
            events = events[:, None]
        assert times.shape[0] > times.shape[1], f'too many events: {times.shape[1]} events but only {times.shape[0]} observations'
        
        self.model_ = GBSURV(**self.model_params, device=torch_device)
        self.model_.fit(X, times, events)
        
        return self

    def predict(self, X, eval_times):
        return self.model_.predict(X, eval_times)[:, 0]

In [42]:
class GBMInter(GBM):
    short_name = 'gbmi'
    long_name = 'gbmi'
    multi = True
    competing = False
    
    @staticmethod
    def trial2model_params(trial):
        p = GBM.trial2model_params(trial)
        p['interaction_size'] = trial.suggest_int('interaction_size_log2', 2, 6)
        return p

### Albero/Cox (Piero)

In [43]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.base import clone
from sklearn.linear_model import LinearRegression

In [44]:
# qui ho provato a fare una versione
from dataclasses import dataclass
@dataclass
class PieroReg:
    max_depth: int = None
    min_samples_leaf: int = 10
    base_regressor: object = LinearRegression()

    def fit(self, X, y):
        self.classifier_ = DecisionTreeRegressor(max_depth=self.max_depth, min_samples_leaf=self.min_samples_leaf)
        self.classifier_.fit(X, y)
        clusters = self.classifier_.apply(X)
        self.regressors_ = {}
        for c in numpy.unique(clusters):
            idx = clusters == c
            self.regressors_[c] = clone(self.base_regressor).fit(X[idx], y[idx])
        return self

    def predict(self, X):
        pred = numpy.full(len(X), numpy.nan)
        clusters = self.classifier_.apply(X)
        for c in numpy.unique(clusters):
            idx = clusters == c
            pred[idx] = self.regressors_[c].predict(X[idx])
        return pred


In [45]:
from sksurv.linear_model import CoxnetSurvivalAnalysis
from dataclasses import dataclass
@dataclass
class PieroCox:
    max_depth: int = None
    min_samples_leaf: int = 10
    alpha: float = 1.0
    l1_ratio: float = 0.5

    def fit(self, X, y):
        self.classifier_ = DecisionTreeRegressor(max_depth=self.max_depth, min_samples_leaf=self.min_samples_leaf)
        self.classifier_.fit(X[y['event']], y['time'][y['event']])
        clusters = self.classifier_.apply(X)
        self.regressors_ = {}
        for c in numpy.unique(clusters):
            idx = clusters == c
            self.regressors_[c] = CoxnetSurvivalAnalysis(alphas=[self.alpha], l1_ratio=self.l1_ratio, fit_baseline_model=True).fit(X[idx], y[idx])
        return self

    def predict_survival_function(self, X):
        pred = numpy.full(len(X), None)
        clusters = self.classifier_.apply(X)
        for c in numpy.unique(clusters):
            idx = clusters == c
            pred[idx] = self.regressors_[c].predict_survival_function(X[idx])
        return pred

In [46]:
class PieroMethod(SurvMethod):
    short_name = 'piero'
    long_name = 'Albero/Cox (Piero)'
    competing = False
    multi = False
    
    @staticmethod
    def trial2model_params(trial):
        return dict(            
            max_depth = trial.suggest_int('max_depth', 1, 6),
            min_samples_leaf = 2**trial.suggest_int('min_samples_leaf_log2', 2, 8),
            alpha=trial.suggest_float('alpha', 0.05, 10),
            l1_ratio=trial.suggest_float('l1_ratio', 0, 1),
            #ties=trial.suggest_categorical('ties', ["breslow", "efron"]),
            #n_iter=10000, tol=1e-09, verbose=0,
        )

    def fit(self, X, times, events):
        y = surv_vector(times, events)
        try:
            self.model_ = PieroCox(**self.model_params).fit(X, y)
        except ArithmeticError as e:
            raise FailedModel(f'{self.short_name} model fit failed: {e}')
        return self
    
    def predict(self, X, eval_times):
        """predict probabilites of event up to given times for each event"""
        def sfext(sf, t):
            if t <= sf.x[0]:
                return 1.0
            if t < sf.x[-1]:
                return sf(t)
            return 0.0
        return numpy.array([[1 - sfext(sf, t) for t in eval_times] for sf in self.model_.predict_survival_function(X)])

### SODEN

In [47]:
if False:
    import sys
    sys.path.insert(0, './torchdiffeq')
    sys.path.insert(0, './lifelines')
    sys.path.insert(0, './autograd')
    sys.path.insert(0, './formulaic')
    sys.path.insert(0, './interface_meta')
    sys.path.insert(0, './astor')
    sys.path.insert(0, './autograd-gamma')
    sys.path.append('./SODEN')
    from SODEN.models import SODENModel

In [48]:
#model = SODENModel(
#    model_config=model_config, feature_size=feature_size, use_embed=use_embed)
#model.to(args.device)

## RUN

In [49]:
base_dir = '../brainteaser/survival-2022-05/'
work_dir = 'workdir'

In [50]:
def make_competing(times, main_event, competing_event):
    combined_event = main_event.astype(float)
    combined_event.loc[(main_event == 0) & (competing_event == 1)] = 2
    return times, combined_event

def preprocess_outcomes():
    df = pandas.read_csv(f'{base_dir}/events.csv', index_col='id')
    
    # fix zero times, they can be problematic for various methods
    time_cols = [c for c in df.columns if c.endswith('_time')]
    min_times = df[time_cols].replace(0.0, numpy.nan).min()
    df[time_cols] = df[time_cols].replace(0.0, min_times/2)
    assert (df[time_cols] > 0).all().all()
    events = ['death', 'NIV', 'PEG']
    
    multi = {}
    for i, evt in enumerate(events): 
        sorted_events = events[i:] + events[:i] # put main event first
        multi[evt] = (
            df[[e + '_time' for e in sorted_events]],
            df[[e + '_event' for e in sorted_events]].astype(bool),
        )
        
    return {
        'independent': {evt: (df[f'{evt}_time'], df[f'{evt}_event'].astype(bool)) for evt in events},
        'competing': {evt: make_competing(df[f'{evt}_time'], df[f'{evt}_event'], df.death_event) for evt in ['NIV', 'PEG']},
        'multi': multi,
    }
assert all(events.max() == 2 for evt, (times, events) in preprocess_outcomes()['competing'].items())

In [51]:
def test_methods(methods=[SkCoxPH, SkRSF, DeepHitMethod, DeepSurvivalMachines, SurvTraceMethod, GBM, PieroMethod], event_mode=None):
    Xdiag = pandas.read_csv(f'{base_dir}/Xdiag.csv', index_col=0)
    outcomes = preprocess_outcomes()

    acc = {}
    for method in methods:
        for outtype, ys in outcomes.items():
            if event_mode is None:
                do = outtype == 'independent' or getattr(method, outtype)
            else:
                do = outtype == event_mode
            if do:
                for evt, y in ys.items():
                    #print(method.long_name, outtype, evt)
                    acc[(method.long_name, outtype, evt)] = optimize_model(
                        method, Xdiag.iloc[:300], *(yy[:300] for yy in y), max_trials=1, 
                        study_name=None)
    return acc

In [52]:
import multiprocessing
def run_par_func(args):
    Xdiag, y, seed, method, outtype, evt, skip_eval = args
    key = (seed, method.long_name, outtype, evt)
    return key, optimize_model(
        method, Xdiag, *y, max_trials=trials, 
        study_name=f'{method.short_name}_diag_{outtype}_{evt}_{seed}', seed=seed, skip_eval=skip_eval)

    
def run_par(methods, seeds=1, trials=10, skip_eval=False, n_jobs=0):
    Xdiag = pandas.read_csv(f'{base_dir}/Xdiag.csv', index_col=0)
    outcomes = preprocess_outcomes()
    if isinstance(seeds, int):
        seeds = list(range(seeds))
    
    tasks = []
    for seed in seeds:
        for method in methods:
            for outtype, ys in outcomes.items():
                if outtype == 'independent' or getattr(method, outtype):
                    for evt, y in ys.items():
                        tasks.append((Xdiag, y, seed, method, outtype, evt, skip_eval))

    metric_acc = {}
    trial_acc = {}
    if n_jobs > 0:
        with multiprocessing.Pool(n_jobs) as p:
            for key, values in p.imap_unordered(run_par_func, tasks):
                print(key)
                metric_acc[key], trial_acc[key] = values
    else:
        for key, values in map(run_par_func, tasks):
            print(key)
            metric_acc[key], trial_acc[key] = values


    return metric_acc, trial_acc

    trial_df = pandas.concat({k: pandas.Series(c) for k, c in trial_acc.items()})
    empty_scores = {k: s for k, s in metric_acc.items() if len(s) == 0}
    ok_scores = {k: s for k, s in metric_acc.items() if len(s) > 0}
    if empty_scores:
        print('Empty scores:', empty_scores)
        metric_acc = {k: s for k, s in metric_acc.items() if len(s) > 0}
    #try:
    #    eval_df = None if skip_eval else pandas.concat(metric_acc, names=['SEED', 'METHOD', 'MODE', 'EVENT'])
    #except Exception as e:
    #    print(f'Error {type(e)}: {e}')
    #    eval_df = metric_acc
    
    return eval_df, trial_df

In [53]:
try:
    get_ipython()
    script = False
except NameError:
    script = True

import argparse
parser = argparse.ArgumentParser()
parser.add_argument('--trials', nargs='+', type=int)
parser.add_argument('--seeds', nargs='+', default=list(range(10)), type=int)
parser.add_argument('--models', choices=['CUDA', 'CPU', 'BOTH', 'AUTO'], default='AUTO')
parser.add_argument('--skip-eval', action='store_true', default=False)
parser.add_argument('--output', default='als benchmark results.csv')
parser.add_argument('--cores', type=int, default=0)
parser.add_argument('--device')

args = parser.parse_args(None if script else [])
print('args:', args)

args: Namespace(trials=None, seeds=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], models='AUTO', skip_eval=False, output='als benchmark results.csv', cores=0, device=None)


In [54]:
method_lists = dict(
    CUDA = [DeepHitMethod, SurvTraceMethod, GBM, DeepHitMethodRes, DeepSurvMethod, GBMInter],
    CPU  = [SkCoxPH, DeepSurvivalMachines, PieroMethod],
)
method_lists['ALL'] = [m for methods in method_lists.values() for m in methods]

In [55]:
import torch
if args.device is None:
    if torch.cuda.is_available():
        torch_device = 'cuda:0'
    else:
        torch_device = 'cpu'
else:
    torch_device = args.device

detected_device = 'CUDA' if torch_device.startswith('cuda') else 'CPU'
print(detected_device, 'detected, torch device:', torch_device)

CPU detected, torch device: cpu


In [ ]:
out = sys.stdout
if args.trials is None:
    selected_methods = method_lists['ALL']
    max_trials = [0]
else:
    selected_methods = method_lists[detected_device if args.models == 'AUTO' else args.models]
    max_trials = args.trials
for trials in max_trials:
    print('Required trials:', trials, file=out)
    eacc, tacc = run_par(selected_methods, seeds=args.seeds, trials=trials, skip_eval=args.skip_eval, n_jobs=args.cores)

Required trials: 0


[I 2023-01-20 11:50:46,509] Using an existing study with name 'deephit_diag_independent_death_0' instead of creating a new one.


Loaded 102 trials: 100 COMPLETE, 2 FAIL


[I 2023-01-20 11:50:47,480] Using an existing study with name 'deephit_diag_independent_NIV_0' instead of creating a new one.


(0, 'DeepHit', 'independent', 'death')
Loaded 104 trials: 100 COMPLETE, 1 PRUNED, 3 FAIL


...

[I 2023-01-20 11:50:49,471] Using an existing study with name 'deephit_diag_competing_PEG_0' instead of creating a new one.


(0, 'DeepHit', 'competing', 'NIV')
Loaded 100 trials: 100 COMPLETE


[I 2023-01-20 11:50:50,149] Using an existing study with name 'survtrace_diag_independent_death_0' instead of creating a new one.


(0, 'DeepHit', 'competing', 'PEG')
Loaded 107 trials: 6 FAIL, 100 COMPLETE, 1 RUNNING
Error loading model (will be retrained): Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.
GPU not found! will use cpu for training!


/archive/home/gbirolo/projects/als-surv-benchmarks/./SurvTRACE/survtrace/utils.py:78: UserWarning: Got event/censoring at start time. Should be removed! It is set s.t. it has no contribution to loss.
  warnings.warn("""Got event/censoring at start time. Should be removed! It is set s.t. it has no contribution to loss.""")
/archive/home/gbirolo/projects/als-surv-benchmarks/./SurvTRACE/survtrace/train_utils.py:208: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at /pytorch/torch/csrc/utils/python_arg_parser.cpp:1420.)
  next_m.mul_(beta1).add_(1 - beta1, grad)


In [ ]:
trial_df = pandas.concat({k: pandas.Series(c, dtype=int) for k, c in tacc.items()}).reset_index()
trial_df.columns = ['SEED', 'METHOD', 'MODE', 'EVENT', 'STATE', 'COUNT']

In [ ]:
trial_df.query('STATE == "COMPLETE"').groupby(['METHOD', 'MODE', 'EVENT', 'STATE', 'COUNT']).nunique()

In [ ]:
if args.skip_eval:
    print('ALL DONE')
    sys.exit(0)

In [ ]:
eval_df = pandas.concat({k: s for k, s in eacc.items() if len(s) > 0}, names=['SEED', 'METHOD', 'MODE', 'EVENT']).unstack('METRIC')

In [ ]:
# an evaluation for each of 10 seeds
assert (eval_df.groupby(['METHOD', 'MODE', 'EVENT', 'TIME']).count() == 10).all().all()

In [ ]:
eval_df.head()

In [ ]:
eval_df.to_csv('eval_df_cache.csv')

In [ ]:
eval_df = pandas.read_csv('eval_df_cache.csv', index_col=['SEED', 'METHOD', 'MODE', 'EVENT', 'ITIME', 'TIME'])
eval_df.columns.name = 'METRIC'
eval_df.head()

In [ ]:
def eval_score(score):
    return eval_df[score].reset_index().drop(
        ['SEED', 'TIME'], axis=1).groupby(['METHOD', 'MODE', 'EVENT']).agg(
        ['mean', 'min', 'max', 'count']).sort_values((score, 'mean'))

In [ ]:
import seaborn
seaborn.set_style("whitegrid")
def plot_score(score):
    data = eval_df[score].reset_index()
    plot = seaborn.catplot(
        data=data,
        x='METHOD',
        y=score,
        hue='MODE',
        row='EVENT',
        kind='box',
        aspect=2,
        sharey=False,
    )
    plt.xticks(rotation=90)
    return plot
def plot_score_times(score):
    data = eval_df[score].reset_index()
    data['METHOD-MODE'] = data['METHOD'] + '\n' + data['MODE']
    plot = seaborn.catplot(
        data=data,
        x='METHOD-MODE',
        y=score,
        hue='ITIME',
        row='EVENT',
        kind='box',
        aspect=2,
        sharey=False,
    )
    plt.xticks(rotation=90)
    return plot

### concordance_index_censored

In [ ]:
plot_score('concordance_index_censored')

In [ ]:
plot_score_times('concordance_index_censored')

In [ ]:
eval_score('concordance_index_censored')

### concordance_index_ipcw

In [ ]:
plot_score('concordance_index_ipcw')

In [ ]:
plot_score_times('concordance_index_ipcw')

In [ ]:
eval_score('concordance_index_ipcw')

### cumulative_dynamic_auc

In [ ]:
plot_score('cumulative_dynamic_auc')

In [ ]:
plot_score_times('cumulative_dynamic_auc')

In [ ]:
eval_score('cumulative_dynamic_auc')

### raw_brier_score

In [ ]:
plot_score('raw_brier_score')

In [ ]:
plot_score_times('raw_brier_score')

In [ ]:
eval_score('raw_brier_score')

## best hyperparams

In [ ]:
seed = 0
outtype = 'independent'
evt = 'death'
method = GBM
acc = {}
for seed in range(10):
    study_name = f'{method.short_name}_diag_{outtype}_{evt}_{seed}'
    study = optuna.create_study(
        direction='maximize', 
        sampler=optuna.samplers.RandomSampler(seed=seed), 
        study_name=study_name, 
        load_if_exists=True,
        storage=f'sqlite:///{work_dir}/{study_name}.optuna.sqlite3',
    )
    acc[seed] = study.trials_dataframe()


In [ ]:
df = pandas.concat(acc, names=['seed', 'trial']).reset_index()
param_cols = {c: c[7:] for c in df.columns if c.startswith('params_')}
df = df.rename(columns=param_cols).rename(columns={'value': 'c-index'})
df

In [ ]:
df['c-index'].plot.hist()
#df[plot_cols.keys()].rename(columns=plot_cols).sort_values('c-index', ascending=False)

In [ ]:
param_cols.values()

In [ ]:
df['c-index-interval'] = pandas.qcut(df['c-index'], [0.0, 0.5, 0.75, 0.9, 1.0]).astype(str)
lf = df.set_index('c-index-interval')[param_cols.values()].stack().reset_index()
lf.columns = 'c-index-interval', 'param', 'value'
lf

In [ ]:
df[['c-index-interval', p]].value_counts().unstack(p)

In [ ]:
int_order = sorted(df['c-index-interval'].unique())
for p in param_cols.values():
    try:
        seaborn.boxplot(data=df, x='c-index-interval', y=p, order=int_order)
    except TypeError:
        m = df[['c-index-interval', p]].value_counts().unstack(p).loc[int_order]
        seaborn.heatmap(m, annot=True, fmt='d')
    plt.title(p)
    plt.show()
    plt.close()

In [ ]:
seaborn.catplot(data=lf, x='c-index-interval', y='value', col='param', sharey=False, col_wrap=4, kind='box')

In [ ]:
#plt.figure(figsize=(15, 7))
optuna.visualization.matplotlib.plot_parallel_coordinate(study)

In [ ]:
    study = optuna.create_study(
        direction='maximize', sampler=optuna.samplers.RandomSampler(seed=seed), study_name=study_name, load_if_exists=True,
        storage=(f'sqlite:///{work_dir}/{study_name}.optuna.sqlite3' if study_name is not None else None))


In [ ]:
if script:
    print('ALL DONE')
    sys.exit(0)

In [ ]:
assert False, 'STOP HERE'

## TESTS

In [ ]:
def f():
    Xdev, time_dev, event_dev = test_data
    from sksurv.nonparametric import ipc_weights
    evt = 0
    return ipc_weights(event_dev[:, evt], time_dev[:, evt])
#seaborn.displot(f())
numpy.sort(f())

In [ ]:
def f():
    Xdev, time_dev, event_dev = test_data
    preds = m.predict(Xdev, eval_times_dev)
    t= 36
    raw = [[sf(t) for sf in m.predict_survival_function(Xdev)] for m in m.models_]
    #sf = m.models_[0].predict_cumulative_hazard_function(Xdev)[1]
    raw = [[1 - sf(t) for sf in m.predict_survival_function(Xdev)] for m in m.models_]
    return raw
    #numpy.mean(numpy.square(preds[:, evt]
    for evt in range(time_dev.shape[1]):
        for ti, t in enumerate(eval_times_dev[evt]):
            keep = (time_dev[:, evt] > t) | event_dev[:, evt]
            #print(numpy.mean(keep))
            p = preds[keep, evt, ti]
            y = time_dev[keep] > t
            score = numpy.mean(numpy.square(p - y))
            break
    return score
max(f()[0])

In [ ]:
m.models_[0].predict_cumulative_hazard_function(Xtmp)[2](2)

In [ ]:
numpy.array([[[sf(t) for t in [3,7, 12, 18, 36]] for sf in mm.predict_cumulative_hazard_function(Xtmp)] for mm in m.models_]).shape

In [ ]:
numpy.array([[numpy.interp([3,7, 18], sf.x, sf.y) for sf in mm.predict_cumulative_hazard_function(Xtmp)] for mm in m.models_])

In [ ]:
#preds = m.models_[0].predict_cumulative_hazard_function(Xtmp)#[0].y.shape
[numpy.interp([3,7], sf.x, sf.y) for sf in m.models_[0].predict_cumulative_hazard_function(Xtmp)]

In [ ]:
study.best_trial

In [ ]:
#SkCoxPH(study.best_trial).predict(Xdiag

In [ ]:
#from sksurv.metrics import concordance_index_ipcw, brier_score, cumulative_dynamic_auc, concordance_index_censored


In [ ]:
[k for k, v in globals().items() if k[0] != '_' and type(v) != type(numpy)]